In [41]:
import os
import shutil
import codecs
import chardet
import json

def run_txt(input_dir):
    run = 1
    for file in os.listdir(input_dir):
        os.rename(f'./defect/{file}', f'./defect/defect{run}.txt')
        run += 1

def run_tojson(input_dir, output_file):
    for file in os.listdir(input_dir):
        with open(f'./defect/{file}', 'r') as input_file:
            run_dict = input_file.readlines()
            with open(output_file, 'a') as output_json:
                json.dump(run_dict, output_json)

run_tojson('./defect/', './defect_runs.json')






{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
